In [64]:
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from tqdm import tqdm
from scipy.spatial import distance
from numpy.linalg import norm


In [2]:
doc1 = pd.read_excel('pca.xlsx')
doc2 = pd.read_excel('dimensionality_redution.xlsx')
doc3 = pd.read_excel('food.xlsx')
print(len(doc1))
print(len(doc2))
print(len(doc3))


5
7
4


In [3]:
def text_preprocessing(
        text: list,
        punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_“~''',
        stop_words=['and', 'a', 'is', 'the', 'in','andor', 'be','he','their','usually','within','useds','unique','used','will', 'an','was', 'but', 'this', 'were', 'with', 'of', 'also','on', '.', 'for', 'any', 'its', 'and', 'are', 'from', 'both', 'as','or','into','some','that','so','to','can','such','other','has','per','while','by','most','where','than','use','two']
) -> list:
    """
    A method to preproces text
    """
    for x in text.lower():
        if x in punctuations:
            text = text.replace(x, "")

    # Removing words that have numbers in them
    text = re.sub(r'\w*\d\w*', '', text)

    # Removing digits
    text = re.sub(r'[0-9]+', '', text)

    # Cleaning the whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Setting every word to lower
    text = text.lower()

    # Converting all our text to a list 
    text = text.split(' ')

    # Droping empty strings
    text = [x for x in text if x != '']

    # Droping stop words
    text = [x for x in text if x not in stop_words]

    return text



In [4]:
# Defining the window for context
window = 2

# Creating a placeholder for the scanning of the word list
word_lists_doc1 = []
all_text_doc1 = []
for text in doc1['text']:

    # Cleaning the text
    text = text_preprocessing(text)
    print (text)
    
    # Appending to the all text list
    all_text_doc1 += text

    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text):
                word_lists_doc1.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words
            if i - w - 1 >= 0:
                word_lists_doc1.append([word] + [text[(i - w - 1)]])
                
word_lists_doc1

['principal', 'component', 'analysis', 'pca', 'popular', 'technique', 'analyzing', 'large', 'datasets', 'containing', 'high', 'number', 'dimensionsfeatures', 'observation']
['increasing', 'interpretability', 'data', 'preserving', 'maximum', 'amount', 'information', 'enabling', 'visualization', 'multidimensional', 'data', 'formally']
['pca', 'statistical', 'technique', 'reducing', 'dimensionality', 'dataset', 'accomplished', 'linearly', 'transforming', 'data', 'new', 'coordinate', 'system', 'variation', 'data', 'described', 'fewer', 'dimensions', 'initial', 'data']
['many', 'studies', 'first', 'principal', 'components', 'order', 'plot', 'data', 'dimensions', 'visually', 'identify', 'clusters', 'closely', 'related', 'data', 'points']
['principal', 'component', 'analysis', 'applications', 'many', 'fields', 'population', 'genetics', 'microbiome', 'studies', 'atmospheric', 'science']


[['principal', 'component'],
 ['principal', 'analysis'],
 ['component', 'analysis'],
 ['component', 'principal'],
 ['component', 'pca'],
 ['analysis', 'pca'],
 ['analysis', 'component'],
 ['analysis', 'popular'],
 ['analysis', 'principal'],
 ['pca', 'popular'],
 ['pca', 'analysis'],
 ['pca', 'technique'],
 ['pca', 'component'],
 ['popular', 'technique'],
 ['popular', 'pca'],
 ['popular', 'analyzing'],
 ['popular', 'analysis'],
 ['technique', 'analyzing'],
 ['technique', 'popular'],
 ['technique', 'large'],
 ['technique', 'pca'],
 ['analyzing', 'large'],
 ['analyzing', 'technique'],
 ['analyzing', 'datasets'],
 ['analyzing', 'popular'],
 ['large', 'datasets'],
 ['large', 'analyzing'],
 ['large', 'containing'],
 ['large', 'technique'],
 ['datasets', 'containing'],
 ['datasets', 'large'],
 ['datasets', 'high'],
 ['datasets', 'analyzing'],
 ['containing', 'high'],
 ['containing', 'datasets'],
 ['containing', 'number'],
 ['containing', 'large'],
 ['high', 'number'],
 ['high', 'containing'],

In [5]:
# Defining the window for context
window = 2

# Creating a placeholder for the scanning of the word list
word_lists_doc2 = []
all_text_doc2 = []
for text in doc2['text']:

    # Cleaning the text
    text = text_preprocessing(text)
    print (text)
    
    # Appending to the all text list
    all_text_doc2 += text

    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text):
                word_lists_doc2.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words
            if i - w - 1 >= 0:
                word_lists_doc2.append([word] + [text[(i - w - 1)]])
                
word_lists_doc2

['dimensionality', 'reduction', 'dimension', 'reduction', 'transformation', 'data', 'highdimensional', 'space', 'lowdimensional', 'space', 'lowdimensional', 'representation', 'retains', 'meaningful', 'properties', 'original', 'data']
['ideally', 'close', 'intrinsic', 'dimension', 'working', 'highdimensional', 'spaces', 'undesirable', 'many', 'reasons', 'raw', 'data', 'often', 'sparse', 'consequence', 'curse', 'dimensionality']
['analyzing', 'data', 'computationally', 'intractable', 'hard', 'control', 'deal']
['dimensionality', 'reduction', 'common', 'fields', 'deal', 'large', 'numbers', 'observations', 'large', 'numbers', 'variables']
['signal', 'processing', 'speech', 'recognition', 'neuroinformatics', 'bioinformatics', 'methods', 'commonly', 'divided', 'linear', 'nonlinear', 'approaches']
['approaches', 'divided', 'feature', 'selection', 'feature', 'extraction']
['dimensionality', 'reduction', 'noise', 'reduction', 'data', 'visualization', 'cluster', 'analysis', 'intermediate', 'step

[['dimensionality', 'reduction'],
 ['dimensionality', 'dimension'],
 ['reduction', 'dimension'],
 ['reduction', 'dimensionality'],
 ['reduction', 'reduction'],
 ['dimension', 'reduction'],
 ['dimension', 'reduction'],
 ['dimension', 'transformation'],
 ['dimension', 'dimensionality'],
 ['reduction', 'transformation'],
 ['reduction', 'dimension'],
 ['reduction', 'data'],
 ['reduction', 'reduction'],
 ['transformation', 'data'],
 ['transformation', 'reduction'],
 ['transformation', 'highdimensional'],
 ['transformation', 'dimension'],
 ['data', 'highdimensional'],
 ['data', 'transformation'],
 ['data', 'space'],
 ['data', 'reduction'],
 ['highdimensional', 'space'],
 ['highdimensional', 'data'],
 ['highdimensional', 'lowdimensional'],
 ['highdimensional', 'transformation'],
 ['space', 'lowdimensional'],
 ['space', 'highdimensional'],
 ['space', 'space'],
 ['space', 'data'],
 ['lowdimensional', 'space'],
 ['lowdimensional', 'space'],
 ['lowdimensional', 'lowdimensional'],
 ['lowdimensiona

In [6]:
# Defining the window for context
window = 2

# Creating a placeholder for the scanning of the word list
word_lists_doc3 = []
all_text_doc3 = []
for text in doc3['text']:

    # Cleaning the text
    text = text_preprocessing(text)
    print (text)
    
    # Appending to the all text list
    all_text_doc3 += text

    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text):
                word_lists_doc3.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words
            if i - w - 1 >= 0:
                word_lists_doc3.append([word] + [text[(i - w - 1)]])
                
word_lists_doc3

['food', 'substance', 'consumed', 'organism', 'nutritional', 'support']
['food', 'plant', 'animal', 'fungal', 'origin', 'contains', 'essential', 'nutrients', 'carbohydrates', 'fats', 'proteins', 'vitamins', 'minerals']
['substance', 'ingested', 'organism', 'assimilated', 'organisms', 'cells', 'provide', 'energy', 'maintain', 'life', 'stimulate', 'growth']
['different', 'species', 'animals', 'have', 'different', 'feeding', 'behaviours', 'satisfy', 'needs', 'metabolisms', 'often', 'evolved', 'fill', 'specific', 'ecological', 'niche', 'specific', 'geographical', 'contexts']


[['food', 'substance'],
 ['food', 'consumed'],
 ['substance', 'consumed'],
 ['substance', 'food'],
 ['substance', 'organism'],
 ['consumed', 'organism'],
 ['consumed', 'substance'],
 ['consumed', 'nutritional'],
 ['consumed', 'food'],
 ['organism', 'nutritional'],
 ['organism', 'consumed'],
 ['organism', 'support'],
 ['organism', 'substance'],
 ['nutritional', 'support'],
 ['nutritional', 'organism'],
 ['nutritional', 'consumed'],
 ['support', 'nutritional'],
 ['support', 'organism'],
 ['food', 'plant'],
 ['food', 'animal'],
 ['plant', 'animal'],
 ['plant', 'food'],
 ['plant', 'fungal'],
 ['animal', 'fungal'],
 ['animal', 'plant'],
 ['animal', 'origin'],
 ['animal', 'food'],
 ['fungal', 'origin'],
 ['fungal', 'animal'],
 ['fungal', 'contains'],
 ['fungal', 'plant'],
 ['origin', 'contains'],
 ['origin', 'fungal'],
 ['origin', 'essential'],
 ['origin', 'animal'],
 ['contains', 'essential'],
 ['contains', 'origin'],
 ['contains', 'nutrients'],
 ['contains', 'fungal'],
 ['essential', 'nutr

In [7]:
def create_unique_word_dict(text: list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    # Creating the dictionary for the unique words
    unique_word_dict = {}
    for i, word in enumerate(words):
        unique_word_dict.update({
            word: i
        })

    return unique_word_dict


In [8]:
unique_word_dict1 = create_unique_word_dict(all_text_doc1)
unique_word_dict2 = create_unique_word_dict(all_text_doc2)
unique_word_dict3 = create_unique_word_dict(all_text_doc3)

# Defining the number of features (unique words)
n_words_doc1 = len(unique_word_dict1)
n_words_doc2 = len(unique_word_dict2)
n_words_doc3 = len(unique_word_dict3)

print(n_words_doc1)
print(n_words_doc2)
print(n_words_doc3)


59
61
45


In [9]:
unique_word_dict1

{'accomplished': 0,
 'amount': 1,
 'analysis': 2,
 'analyzing': 3,
 'applications': 4,
 'atmospheric': 5,
 'closely': 6,
 'clusters': 7,
 'component': 8,
 'components': 9,
 'containing': 10,
 'coordinate': 11,
 'data': 12,
 'dataset': 13,
 'datasets': 14,
 'described': 15,
 'dimensionality': 16,
 'dimensions': 17,
 'dimensionsfeatures': 18,
 'enabling': 19,
 'fewer': 20,
 'fields': 21,
 'first': 22,
 'formally': 23,
 'genetics': 24,
 'high': 25,
 'identify': 26,
 'increasing': 27,
 'information': 28,
 'initial': 29,
 'interpretability': 30,
 'large': 31,
 'linearly': 32,
 'many': 33,
 'maximum': 34,
 'microbiome': 35,
 'multidimensional': 36,
 'new': 37,
 'number': 38,
 'observation': 39,
 'order': 40,
 'pca': 41,
 'plot': 42,
 'points': 43,
 'popular': 44,
 'population': 45,
 'preserving': 46,
 'principal': 47,
 'reducing': 48,
 'related': 49,
 'science': 50,
 'statistical': 51,
 'studies': 52,
 'system': 53,
 'technique': 54,
 'transforming': 55,
 'variation': 56,
 'visualization': 5

In [10]:
unique_word_dict2

{'analyses': 0,
 'analysis': 1,
 'analyzing': 2,
 'approaches': 3,
 'bioinformatics': 4,
 'close': 5,
 'cluster': 6,
 'common': 7,
 'commonly': 8,
 'computationally': 9,
 'consequence': 10,
 'control': 11,
 'curse': 12,
 'data': 13,
 'deal': 14,
 'dimension': 15,
 'dimensionality': 16,
 'divided': 17,
 'extraction': 18,
 'facilitate': 19,
 'feature': 20,
 'fields': 21,
 'hard': 22,
 'highdimensional': 23,
 'ideally': 24,
 'intermediate': 25,
 'intractable': 26,
 'intrinsic': 27,
 'large': 28,
 'linear': 29,
 'lowdimensional': 30,
 'many': 31,
 'meaningful': 32,
 'methods': 33,
 'neuroinformatics': 34,
 'noise': 35,
 'nonlinear': 36,
 'numbers': 37,
 'observations': 38,
 'often': 39,
 'original': 40,
 'processing': 41,
 'properties': 42,
 'raw': 43,
 'reasons': 44,
 'recognition': 45,
 'reduction': 46,
 'representation': 47,
 'retains': 48,
 'selection': 49,
 'signal': 50,
 'space': 51,
 'spaces': 52,
 'sparse': 53,
 'speech': 54,
 'step': 55,
 'transformation': 56,
 'undesirable': 57,


In [11]:
unique_word_dict3

{'animal': 0,
 'animals': 1,
 'assimilated': 2,
 'behaviours': 3,
 'carbohydrates': 4,
 'cells': 5,
 'consumed': 6,
 'contains': 7,
 'contexts': 8,
 'different': 9,
 'ecological': 10,
 'energy': 11,
 'essential': 12,
 'evolved': 13,
 'fats': 14,
 'feeding': 15,
 'fill': 16,
 'food': 17,
 'fungal': 18,
 'geographical': 19,
 'growth': 20,
 'have': 21,
 'ingested': 22,
 'life': 23,
 'maintain': 24,
 'metabolisms': 25,
 'minerals': 26,
 'needs': 27,
 'niche': 28,
 'nutrients': 29,
 'nutritional': 30,
 'often': 31,
 'organism': 32,
 'organisms': 33,
 'origin': 34,
 'plant': 35,
 'proteins': 36,
 'provide': 37,
 'satisfy': 38,
 'species': 39,
 'specific': 40,
 'stimulate': 41,
 'substance': 42,
 'support': 43,
 'vitamins': 44}

In [12]:
all_text=all_text_doc1+all_text_doc2+all_text_doc3

unique_word_dict = create_unique_word_dict(all_text)
# Defining the number of features (unique words)
n_words = len(unique_word_dict)
print(n_words)
unique_word_dict

156


{'accomplished': 0,
 'amount': 1,
 'analyses': 2,
 'analysis': 3,
 'analyzing': 4,
 'animal': 5,
 'animals': 6,
 'applications': 7,
 'approaches': 8,
 'assimilated': 9,
 'atmospheric': 10,
 'behaviours': 11,
 'bioinformatics': 12,
 'carbohydrates': 13,
 'cells': 14,
 'close': 15,
 'closely': 16,
 'cluster': 17,
 'clusters': 18,
 'common': 19,
 'commonly': 20,
 'component': 21,
 'components': 22,
 'computationally': 23,
 'consequence': 24,
 'consumed': 25,
 'containing': 26,
 'contains': 27,
 'contexts': 28,
 'control': 29,
 'coordinate': 30,
 'curse': 31,
 'data': 32,
 'dataset': 33,
 'datasets': 34,
 'deal': 35,
 'described': 36,
 'different': 37,
 'dimension': 38,
 'dimensionality': 39,
 'dimensions': 40,
 'dimensionsfeatures': 41,
 'divided': 42,
 'ecological': 43,
 'enabling': 44,
 'energy': 45,
 'essential': 46,
 'evolved': 47,
 'extraction': 48,
 'facilitate': 49,
 'fats': 50,
 'feature': 51,
 'feeding': 52,
 'fewer': 53,
 'fields': 54,
 'fill': 55,
 'first': 56,
 'food': 57,
 'f

In [13]:
word_lists=word_lists_doc1+word_lists_doc2+word_lists_doc3
word_lists

[['principal', 'component'],
 ['principal', 'analysis'],
 ['component', 'analysis'],
 ['component', 'principal'],
 ['component', 'pca'],
 ['analysis', 'pca'],
 ['analysis', 'component'],
 ['analysis', 'popular'],
 ['analysis', 'principal'],
 ['pca', 'popular'],
 ['pca', 'analysis'],
 ['pca', 'technique'],
 ['pca', 'component'],
 ['popular', 'technique'],
 ['popular', 'pca'],
 ['popular', 'analyzing'],
 ['popular', 'analysis'],
 ['technique', 'analyzing'],
 ['technique', 'popular'],
 ['technique', 'large'],
 ['technique', 'pca'],
 ['analyzing', 'large'],
 ['analyzing', 'technique'],
 ['analyzing', 'datasets'],
 ['analyzing', 'popular'],
 ['large', 'datasets'],
 ['large', 'analyzing'],
 ['large', 'containing'],
 ['large', 'technique'],
 ['datasets', 'containing'],
 ['datasets', 'large'],
 ['datasets', 'high'],
 ['datasets', 'analyzing'],
 ['containing', 'high'],
 ['containing', 'datasets'],
 ['containing', 'number'],
 ['containing', 'large'],
 ['high', 'number'],
 ['high', 'containing'],

In [14]:
# Getting all the unique words
words = list(unique_word_dict.keys())
print(words)

# Creating the X and Y matrices using one hot encoding
print(n_words)
X = []
Y = []
for i, word_list in tqdm(enumerate(word_lists)):
    # Getting the indices
    print(word_list)
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])
    # print (word_list)
    print(word_list[0], main_word_index)
    print(word_list[1], context_word_index)

    # Creating the placeholders
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    # One hot encoding the main word
    X_row[main_word_index] = 1

    # One hot encoding the Y matrix words
    Y_row[context_word_index] = 1

    # Appending to the main matrices
    X.append(X_row)
    Y.append(Y_row)

# Converting the matrices into a sparse format because the vast majority of the data are 0s
print(X)
print(Y)


['accomplished', 'amount', 'analyses', 'analysis', 'analyzing', 'animal', 'animals', 'applications', 'approaches', 'assimilated', 'atmospheric', 'behaviours', 'bioinformatics', 'carbohydrates', 'cells', 'close', 'closely', 'cluster', 'clusters', 'common', 'commonly', 'component', 'components', 'computationally', 'consequence', 'consumed', 'containing', 'contains', 'contexts', 'control', 'coordinate', 'curse', 'data', 'dataset', 'datasets', 'deal', 'described', 'different', 'dimension', 'dimensionality', 'dimensions', 'dimensionsfeatures', 'divided', 'ecological', 'enabling', 'energy', 'essential', 'evolved', 'extraction', 'facilitate', 'fats', 'feature', 'feeding', 'fewer', 'fields', 'fill', 'first', 'food', 'formally', 'fungal', 'genetics', 'geographical', 'growth', 'hard', 'have', 'high', 'highdimensional', 'ideally', 'identify', 'increasing', 'information', 'ingested', 'initial', 'intermediate', 'interpretability', 'intractable', 'intrinsic', 'large', 'life', 'linear', 'linearly', '

728it [00:00, 33373.26it/s]

['principal', 'component']
principal 116
component 21
['principal', 'analysis']
principal 116
analysis 3
['component', 'analysis']
component 21
analysis 3
['component', 'principal']
component 21
principal 116
['component', 'pca']
component 21
pca 109
['analysis', 'pca']
analysis 3
pca 109
['analysis', 'component']
analysis 3
component 21
['analysis', 'popular']
analysis 3
popular 113
['analysis', 'principal']
analysis 3
principal 116
['pca', 'popular']
pca 109
popular 113
['pca', 'analysis']
pca 109
analysis 3
['pca', 'technique']
pca 109
technique 146
['pca', 'component']
pca 109
component 21
['popular', 'technique']
popular 113
technique 146
['popular', 'pca']
popular 113
pca 109
['popular', 'analyzing']
popular 113
analyzing 4
['popular', 'analysis']
popular 113
analysis 3
['technique', 'analyzing']
technique 146
analyzing 4
['technique', 'popular']
technique 146
popular 113
['technique', 'large']
technique 146
large 77
['technique', 'pca']
technique 146
pca 109
['analyzing', 'large

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   

In [15]:
XX = tf.convert_to_tensor(X, dtype=tf.float32)
YY = tf.convert_to_tensor(Y, dtype=tf.float32)
print(XX.shape)
print(YY.shape)


(728, 156)
(728, 156)


2022-11-28 20:43:36.511092: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-11-28 20:43:36.511124: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: abanoub-G3-3590
2022-11-28 20:43:36.511132: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: abanoub-G3-3590
2022-11-28 20:43:36.511250: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.65.1
2022-11-28 20:43:36.511276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.141.3
2022-11-28 20:43:36.511284: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 470.141.3 does not match DSO version 515.65.1 -- cannot find working devices in this configuration
20

In [16]:
def CreateModel():
    # Defining the size of the embedding
    embed_size = 2
    # Defining the neural network

    # inp = Input(shape=(X.shape[1],))
    inp = Input(shape=XX.shape[1])  # 21
    x = Dense(units=embed_size, activation='linear')(inp)
    # x = Dense(units=21, activation='softmax')(x)
    x = Dense(units=YY.shape[1], activation='softmax')(x)

    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.summary()
    return model


In [17]:
model = CreateModel()
# Optimizing the network weights
model.fit(x=XX,y=YY,batch_size=10,epochs=5000)
# The input layer
weights = model.get_weights()[0]

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 156)]             0         
                                                                 
 dense (Dense)               (None, 2)                 314       
                                                                 
 dense_1 (Dense)             (None, 156)               468       
                                                                 
Total params: 782
Trainable params: 782
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5000
73/73 [==============================] - 0s 922us/step - loss: 5.0504
Epoch 2/5000
73/73 [==============================] - 0s 809us/step - loss: 5.0413
Epoch 3/5000
73/73 [==============================] - 0s 822us/step - loss: 5.0343
Epoch 4/5000
73/73 [==============================] - 0s 819us/step - loss: 5.

73/73 [==============================] - 0s 759us/step - loss: 3.9821
Epoch 90/5000
73/73 [==============================] - 0s 755us/step - loss: 3.9751
Epoch 91/5000
73/73 [==============================] - 0s 776us/step - loss: 3.9683
Epoch 92/5000
73/73 [==============================] - 0s 773us/step - loss: 3.9611
Epoch 93/5000
73/73 [==============================] - 0s 720us/step - loss: 3.9543
Epoch 94/5000
73/73 [==============================] - 0s 778us/step - loss: 3.9475
Epoch 95/5000
73/73 [==============================] - 0s 769us/step - loss: 3.9406
Epoch 96/5000
73/73 [==============================] - 0s 741us/step - loss: 3.9346
Epoch 97/5000
73/73 [==============================] - 0s 803us/step - loss: 3.9278
Epoch 98/5000
73/73 [==============================] - 0s 819us/step - loss: 3.9211
Epoch 99/5000
73/73 [==============================] - 0s 745us/step - loss: 3.9152
Epoch 100/5000
73/73 [==============================] - 0s 760us/step - loss: 3.9089
Epoch

73/73 [==============================] - 0s 797us/step - loss: 3.5571
Epoch 186/5000
73/73 [==============================] - 0s 818us/step - loss: 3.5539
Epoch 187/5000
73/73 [==============================] - 0s 789us/step - loss: 3.5519
Epoch 188/5000
73/73 [==============================] - 0s 827us/step - loss: 3.5483
Epoch 189/5000
73/73 [==============================] - 0s 784us/step - loss: 3.5460
Epoch 190/5000
73/73 [==============================] - 0s 795us/step - loss: 3.5427
Epoch 191/5000
73/73 [==============================] - 0s 799us/step - loss: 3.5399
Epoch 192/5000
73/73 [==============================] - 0s 830us/step - loss: 3.5372
Epoch 193/5000
73/73 [==============================] - 0s 865us/step - loss: 3.5345
Epoch 194/5000
73/73 [==============================] - 0s 781us/step - loss: 3.5320
Epoch 195/5000
73/73 [==============================] - 0s 825us/step - loss: 3.5295
Epoch 196/5000
73/73 [==============================] - 0s 791us/step - loss: 3.

73/73 [==============================] - 0s 776us/step - loss: 3.3417
Epoch 282/5000
73/73 [==============================] - 0s 764us/step - loss: 3.3409
Epoch 283/5000
73/73 [==============================] - 0s 787us/step - loss: 3.3387
Epoch 284/5000
73/73 [==============================] - 0s 798us/step - loss: 3.3376
Epoch 285/5000
73/73 [==============================] - 0s 800us/step - loss: 3.3353
Epoch 286/5000
73/73 [==============================] - 0s 788us/step - loss: 3.3336
Epoch 287/5000
73/73 [==============================] - 0s 767us/step - loss: 3.3316
Epoch 288/5000
73/73 [==============================] - 0s 778us/step - loss: 3.3305
Epoch 289/5000
73/73 [==============================] - 0s 784us/step - loss: 3.3289
Epoch 290/5000
73/73 [==============================] - 0s 763us/step - loss: 3.3272
Epoch 291/5000
73/73 [==============================] - 0s 728us/step - loss: 3.3253
Epoch 292/5000
73/73 [==============================] - 0s 816us/step - loss: 3.

73/73 [==============================] - 0s 799us/step - loss: 3.2016
Epoch 378/5000
73/73 [==============================] - 0s 773us/step - loss: 3.2005
Epoch 379/5000
73/73 [==============================] - 0s 803us/step - loss: 3.1990
Epoch 380/5000
73/73 [==============================] - 0s 896us/step - loss: 3.1981
Epoch 381/5000
73/73 [==============================] - 0s 823us/step - loss: 3.1966
Epoch 382/5000
73/73 [==============================] - 0s 880us/step - loss: 3.1958
Epoch 383/5000
73/73 [==============================] - 0s 871us/step - loss: 3.1944
Epoch 384/5000
73/73 [==============================] - 0s 873us/step - loss: 3.1932
Epoch 385/5000
73/73 [==============================] - 0s 892us/step - loss: 3.1916
Epoch 386/5000
73/73 [==============================] - 0s 886us/step - loss: 3.1900
Epoch 387/5000
73/73 [==============================] - 0s 843us/step - loss: 3.1902
Epoch 388/5000
73/73 [==============================] - 0s 865us/step - loss: 3.

...

73/73 [==============================] - 0s 825us/step - loss: 2.5805
Epoch 4565/5000
73/73 [==============================] - 0s 808us/step - loss: 2.5811
Epoch 4566/5000
73/73 [==============================] - 0s 836us/step - loss: 2.5804
Epoch 4567/5000
73/73 [==============================] - 0s 801us/step - loss: 2.5792
Epoch 4568/5000
73/73 [==============================] - 0s 813us/step - loss: 2.5829
Epoch 4569/5000
73/73 [==============================] - 0s 799us/step - loss: 2.5793
Epoch 4570/5000
73/73 [==============================] - 0s 762us/step - loss: 2.5803
Epoch 4571/5000
73/73 [==============================] - 0s 817us/step - loss: 2.5821
Epoch 4572/5000
73/73 [==============================] - 0s 804us/step - loss: 2.5812
Epoch 4573/5000
73/73 [==============================] - 0s 793us/step - loss: 2.5804
Epoch 4574/5000
73/73 [==============================] - 0s 841us/step - loss: 2.5784
Epoch 4575/5000
73/73 [==============================] - 0s 783us/step

73/73 [==============================] - 0s 961us/step - loss: 2.5786
Epoch 4660/5000
73/73 [==============================] - 0s 983us/step - loss: 2.5798
Epoch 4661/5000
73/73 [==============================] - 0s 1ms/step - loss: 2.5800
Epoch 4662/5000
73/73 [==============================] - 0s 782us/step - loss: 2.5785
Epoch 4663/5000
73/73 [==============================] - 0s 785us/step - loss: 2.5793
Epoch 4664/5000
73/73 [==============================] - 0s 768us/step - loss: 2.5783
Epoch 4665/5000
73/73 [==============================] - 0s 767us/step - loss: 2.5793
Epoch 4666/5000
73/73 [==============================] - 0s 791us/step - loss: 2.5786
Epoch 4667/5000
73/73 [==============================] - 0s 803us/step - loss: 2.5783
Epoch 4668/5000
73/73 [==============================] - 0s 816us/step - loss: 2.5784
Epoch 4669/5000
73/73 [==============================] - 0s 817us/step - loss: 2.5783
Epoch 4670/5000
73/73 [==============================] - 0s 817us/step -

73/73 [==============================] - 0s 811us/step - loss: 2.5768
Epoch 4755/5000
73/73 [==============================] - 0s 761us/step - loss: 2.5784
Epoch 4756/5000
73/73 [==============================] - 0s 774us/step - loss: 2.5782
Epoch 4757/5000
73/73 [==============================] - 0s 761us/step - loss: 2.5786
Epoch 4758/5000
73/73 [==============================] - 0s 794us/step - loss: 2.5773
Epoch 4759/5000
73/73 [==============================] - 0s 804us/step - loss: 2.5782
Epoch 4760/5000
73/73 [==============================] - 0s 828us/step - loss: 2.5761
Epoch 4761/5000
73/73 [==============================] - 0s 784us/step - loss: 2.5770
Epoch 4762/5000
73/73 [==============================] - 0s 770us/step - loss: 2.5767
Epoch 4763/5000
73/73 [==============================] - 0s 782us/step - loss: 2.5784
Epoch 4764/5000
73/73 [==============================] - 0s 791us/step - loss: 2.5776
Epoch 4765/5000
73/73 [==============================] - 0s 795us/step

73/73 [==============================] - 0s 865us/step - loss: 2.5747
Epoch 4850/5000
73/73 [==============================] - 0s 815us/step - loss: 2.5769
Epoch 4851/5000
73/73 [==============================] - 0s 847us/step - loss: 2.5759
Epoch 4852/5000
73/73 [==============================] - 0s 866us/step - loss: 2.5773
Epoch 4853/5000
73/73 [==============================] - 0s 830us/step - loss: 2.5759
Epoch 4854/5000
73/73 [==============================] - 0s 879us/step - loss: 2.5748
Epoch 4855/5000
73/73 [==============================] - 0s 871us/step - loss: 2.5739
Epoch 4856/5000
73/73 [==============================] - 0s 875us/step - loss: 2.5769
Epoch 4857/5000
73/73 [==============================] - 0s 882us/step - loss: 2.5762
Epoch 4858/5000
73/73 [==============================] - 0s 881us/step - loss: 2.5757
Epoch 4859/5000
73/73 [==============================] - 0s 859us/step - loss: 2.5747
Epoch 4860/5000
73/73 [==============================] - 0s 841us/step

73/73 [==============================] - 0s 799us/step - loss: 2.5740
Epoch 4945/5000
73/73 [==============================] - 0s 760us/step - loss: 2.5746
Epoch 4946/5000
73/73 [==============================] - 0s 793us/step - loss: 2.5734
Epoch 4947/5000
73/73 [==============================] - 0s 833us/step - loss: 2.5730
Epoch 4948/5000
73/73 [==============================] - 0s 796us/step - loss: 2.5750
Epoch 4949/5000
73/73 [==============================] - 0s 763us/step - loss: 2.5743
Epoch 4950/5000
73/73 [==============================] - 0s 818us/step - loss: 2.5739
Epoch 4951/5000
73/73 [==============================] - 0s 786us/step - loss: 2.5740
Epoch 4952/5000
73/73 [==============================] - 0s 793us/step - loss: 2.5752
Epoch 4953/5000
73/73 [==============================] - 0s 802us/step - loss: 2.5751
Epoch 4954/5000
73/73 [==============================] - 0s 800us/step - loss: 2.5736
Epoch 4955/5000
73/73 [==============================] - 0s 812us/step

In [18]:
print(weights.shape)
print(weights)

(156, 2)
[[-1.58913767e+00  1.66224136e+01]
 [ 4.51157331e+00 -1.23740986e-01]
 [-1.27554140e+01  1.41300666e+00]
 [-2.43223763e+00  1.94496930e+00]
 [-2.70148349e+00  5.33488035e+00]
 [ 2.21965623e+00 -8.25957489e+00]
 [ 1.68987198e+01 -2.44034023e+01]
 [-3.85394549e+00  1.58324897e+00]
 [-1.01142139e+01 -9.47604084e+00]
 [ 2.72696667e+01 -2.29222870e+00]
 [-1.27247505e+01 -2.57487321e+00]
 [ 6.03552818e+00 -9.71864986e+00]
 [-2.87332463e+00 -9.35254383e+00]
 [-6.51947784e+00 -2.40898762e+01]
 [ 2.06161423e+01 -6.40840149e+00]
 [-8.73209834e-01  2.16481590e+00]
 [ 1.14627485e+01  6.88911200e+00]
 [-6.74794018e-01  1.80243766e+00]
 [ 1.81160774e+01  5.85894775e+00]
 [-2.24183607e+00  4.16959715e+00]
 [-5.89231014e+00 -9.82923222e+00]
 [-3.16236639e+00  1.97594237e+00]
 [-4.70584631e-02  2.54163086e-01]
 [-1.89363122e+00  5.15819645e+00]
 [ 8.67140174e-01  1.95094287e-01]
 [ 7.98465443e+00 -2.29054379e+00]
 [-1.51622601e+01  7.98875666e+00]
 [-1.51353598e+00 -2.40260639e+01]
 [-2.009122

In [19]:
# get the weight for each unique word
embedding_dict = {}
for word in words:  # to pick the row of weight of two values for each unique word since weights = 21*2
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
    })

print(embedding_dict)

{'accomplished': array([-1.5891377, 16.622414 ], dtype=float32), 'amount': array([ 4.5115733 , -0.12374099], dtype=float32), 'analyses': array([-12.755414 ,   1.4130067], dtype=float32), 'analysis': array([-2.4322376,  1.9449693], dtype=float32), 'analyzing': array([-2.7014835,  5.3348804], dtype=float32), 'animal': array([ 2.2196562, -8.259575 ], dtype=float32), 'animals': array([ 16.89872 , -24.403402], dtype=float32), 'applications': array([-3.8539455,  1.583249 ], dtype=float32), 'approaches': array([-10.114214,  -9.476041], dtype=float32), 'assimilated': array([27.269667 , -2.2922287], dtype=float32), 'atmospheric': array([-12.7247505,  -2.5748732], dtype=float32), 'behaviours': array([ 6.035528, -9.71865 ], dtype=float32), 'bioinformatics': array([-2.8733246, -9.352544 ], dtype=float32), 'carbohydrates': array([ -6.519478, -24.089876], dtype=float32), 'cells': array([20.616142 , -6.4084015], dtype=float32), 'close': array([-0.87320983,  2.164816  ], dtype=float32), 'closely': arr

In [20]:
# get the weight for each unique word
for word in unique_word_dict1: 
    unique_word_dict1[word]=embedding_dict[word]
for word in unique_word_dict2: 
    unique_word_dict2[word]=embedding_dict[word]
for word in unique_word_dict3: 
    unique_word_dict3[word]=embedding_dict[word]

In [21]:
unique_word_dict1

{'accomplished': array([-1.5891377, 16.622414 ], dtype=float32),
 'amount': array([ 4.5115733 , -0.12374099], dtype=float32),
 'analysis': array([-2.4322376,  1.9449693], dtype=float32),
 'analyzing': array([-2.7014835,  5.3348804], dtype=float32),
 'applications': array([-3.8539455,  1.583249 ], dtype=float32),
 'atmospheric': array([-12.7247505,  -2.5748732], dtype=float32),
 'closely': array([11.462749,  6.889112], dtype=float32),
 'clusters': array([18.116077 ,  5.8589478], dtype=float32),
 'component': array([-3.1623664,  1.9759424], dtype=float32),
 'components': array([-0.04705846,  0.2541631 ], dtype=float32),
 'containing': array([-15.16226  ,   7.9887567], dtype=float32),
 'coordinate': array([ 5.569091, 11.60153 ], dtype=float32),
 'data': array([1.1861961, 1.1244164], dtype=float32),
 'dataset': array([-3.2745724, 22.648577 ], dtype=float32),
 'datasets': array([-12.311459 ,   7.0235524], dtype=float32),
 'described': array([11.857405, 12.515061], dtype=float32),
 'dimensio

In [22]:
unique_word_dict2

{'analyses': array([-12.755414 ,   1.4130067], dtype=float32),
 'analysis': array([-2.4322376,  1.9449693], dtype=float32),
 'analyzing': array([-2.7014835,  5.3348804], dtype=float32),
 'approaches': array([-10.114214,  -9.476041], dtype=float32),
 'bioinformatics': array([-2.8733246, -9.352544 ], dtype=float32),
 'close': array([-0.87320983,  2.164816  ], dtype=float32),
 'cluster': array([-0.674794 ,  1.8024377], dtype=float32),
 'common': array([-2.241836,  4.169597], dtype=float32),
 'commonly': array([-5.89231 , -9.829232], dtype=float32),
 'computationally': array([-1.8936312,  5.1581964], dtype=float32),
 'consequence': array([0.8671402 , 0.19509429], dtype=float32),
 'control': array([-3.2442367,  5.962086 ], dtype=float32),
 'curse': array([0.53251255, 0.6368929 ], dtype=float32),
 'data': array([1.1861961, 1.1244164], dtype=float32),
 'deal': array([-4.633265,  5.165632], dtype=float32),
 'dimension': array([-0.63204974,  2.8776066 ], dtype=float32),
 'dimensionality': array

In [23]:
unique_word_dict3

{'animal': array([ 2.2196562, -8.259575 ], dtype=float32),
 'animals': array([ 16.89872 , -24.403402], dtype=float32),
 'assimilated': array([27.269667 , -2.2922287], dtype=float32),
 'behaviours': array([ 6.035528, -9.71865 ], dtype=float32),
 'carbohydrates': array([ -6.519478, -24.089876], dtype=float32),
 'cells': array([20.616142 , -6.4084015], dtype=float32),
 'consumed': array([ 7.9846544, -2.2905438], dtype=float32),
 'contains': array([ -1.513536, -24.026064], dtype=float32),
 'contexts': array([-2.0091224, -3.875744 ], dtype=float32),
 'different': array([  7.792445, -16.93567 ], dtype=float32),
 'ecological': array([-0.609537 , -3.5002253], dtype=float32),
 'energy': array([22.855574, -9.888631], dtype=float32),
 'essential': array([ -3.7750802, -21.295567 ], dtype=float32),
 'evolved': array([ 0.9694564, -2.641056 ], dtype=float32),
 'fats': array([ -8.64195 , -27.712982], dtype=float32),
 'feeding': array([  9.216567, -12.464506], dtype=float32),
 'fill': array([ 0.5447206

# Approach 1 
### Calculate the Euclidean distance between each word in the first document and every word in the second. 

In [24]:
from scipy.spatial import distance
min_dis1=[]
distance_similarity1=0
for word_doc1 in unique_word_dict1:
    word1=''
    word2=''
    minimum=100000000.0
    for word_doc2 in unique_word_dict2:
        x=distance.euclidean(unique_word_dict1[word_doc1],unique_word_dict2[word_doc2]) 
        if x < minimum:
            minimum=x
            word1=word_doc1
            word2=word_doc2
    distance_similarity1+=minimum
    min_dis1.append((word1,word2,minimum))
min_dis1

[('accomplished', 'space', 6.1698808670043945),
 ('amount', 'often', 3.3870227336883545),
 ('analysis', 'analysis', 0.0),
 ('analyzing', 'analyzing', 0.0),
 ('applications', 'intermediate', 1.1995408535003662),
 ('atmospheric', 'analyses', 3.987997531890869),
 ('closely', 'transformation', 11.752825736999512),
 ('clusters', 'original', 16.005414962768555),
 ('component', 'fields', 0.6452721357345581),
 ('components', 'reasons', 0.24930340051651),
 ('containing', 'large', 3.6164844036102295),
 ('coordinate', 'noise', 8.9552640914917),
 ('data', 'data', 0.0),
 ('dataset', 'space', 6.645912170410156),
 ('datasets', 'large', 1.3069709539413452),
 ('described', 'original', 8.625188827514648),
 ('dimensionality', 'dimensionality', 0.0),
 ('dimensions', 'transformation', 7.910202980041504),
 ('dimensionsfeatures', 'extraction', 10.536009788513184),
 ('enabling', 'sparse', 1.3361473083496094),
 ('fewer', 'transformation', 8.411900520324707),
 ('fields', 'fields', 0.0),
 ('first', 'many', 1.382

In [25]:
min_dis2=[]
distance_similarity2=0
for word_doc1 in unique_word_dict1:
    word1=''
    word2=''
    minimum=100000000.0
    for word_doc3 in unique_word_dict3:
        x=distance.euclidean(unique_word_dict1[word_doc1],unique_word_dict3[word_doc3]) 
        if x<minimum:
            minimum=x
            word1=word_doc1
            word2=word_doc2
    distance_similarity2 +=minimum
    min_dis2.append((word1,word2,minimum))
min_dis2

[('accomplished', 'working', 17.559181213378906),
 ('amount', 'working', 3.3870227336883545),
 ('analysis', 'working', 4.485028266906738),
 ('analyzing', 'working', 7.188478469848633),
 ('applications', 'working', 5.532088279724121),
 ('atmospheric', 'working', 10.79430103302002),
 ('closely', 'working', 9.568883895874023),
 ('clusters', 'working', 11.373254776000977),
 ('component', 'working', 5.107336044311523),
 ('components', 'working', 1.5628496408462524),
 ('containing', 'working', 17.713594436645508),
 ('coordinate', 'working', 13.07858657836914),
 ('data', 'working', 1.8419597148895264),
 ('dataset', 'working', 23.786361694335938),
 ('datasets', 'working', 14.997759819030762),
 ('described', 'working', 15.128870964050293),
 ('dimensionality', 'working', 4.803893089294434),
 ('dimensions', 'working', 7.66530179977417),
 ('dimensionsfeatures', 'working', 25.842390060424805),
 ('enabling', 'working', 1.581800937652588),
 ('fewer', 'working', 9.004805564880371),
 ('fields', 'workin

In [26]:
print("distance between document1 and document2 is: ",round(distance_similarity1,2))
print("distance between document1 and document3 is: ",round(distance_similarity2,2))

distance between document1 and document2 is:  281.67
distance between document1 and document3 is:  553.09


# Approach 2
### Calculate the Centroids for each document. 

In [60]:
def get_centroid(document):
    centroids=[]
    for word in document:
        centroids.append((document[word][0],document[word][1]))
    return np.mean(centroids, axis=0)

In [62]:
centroids_doc1=get_centroid(unique_word_dict1)
centroids_doc2=get_centroid(unique_word_dict2)
centroids_doc3=get_centroid(unique_word_dict3)
print("centroids document 1: ",centroids_doc1)
print("centroids document 2: ",centroids_doc2)
print("centroids document 3: ",centroids_doc3)

centroids document 1:  [-1.4807436  5.112427 ]
centroids document 2:  [-3.0727026  2.8814273]
centroids document 3:  [  6.6387854 -10.556029 ]


### calculate cosine similarity between (doc1 and doc2) | and between (doc1 and doc3)  

In [66]:
# compute cosine similarity
similarity1 = np.dot(centroids_doc1,centroids_doc2)/(norm(centroids_doc1)*norm(centroids_doc2))
similarity2 = np.dot(centroids_doc1,centroids_doc3)/(norm(centroids_doc1)*norm(centroids_doc3))
print("similarity between document1 and document2 is: ",round(similarity1,2))
print("similarity between document1 and document3 is: ",round(similarity2,2))

similarity between document1 and document2 is:  0.86
similarity between document1 and document3 is:  -0.96


### Calculate Euclidean distance between (doc1 and doc2) | and between (doc1 and doc3)

In [67]:
# compute cosine similarity
distance1 =distance.euclidean(centroids_doc1,centroids_doc2)
distance2 = distance.euclidean(centroids_doc1,centroids_doc3)
print("distance between document1 and document2 is: ",round(distance1,2))
print("distance between document1 and document3 is: ",round(distance2,2))

distance between document1 and document2 is:  2.74
distance between document1 and document3 is:  17.65
